# How to Train YOLOX on Custom Objects

This tutorial is based on the [YOLOX repository](https://github.com/Megvii-BaseDetection/YOLOX) by [the Megvii Team](https://github.com/Megvii-BaseDetection). This notebook shows training on **your own custom objects**. Many thanks to the Megvii Team for putting this repository together - we hope that in combination with clean data management tools at Roboflow, this technologoy will become easily accessible to any developer wishing to use computer vision in their projects.

### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [How to Train YOLOX](blog.roboflow.com/how-to-train-yolox-on-a-custom-dataset/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train YOLOR on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data. We will use Roboflow to preprocess our images.

To train our detector we take the following steps:

* Install YOLOX dependencies
* Download and Prepare custom YOLOX object detection data
* Download Pre-Trained Weights for YOLOX
* Run YOLOX training
* Evaluate YOLOX performance
* Run YOLOX inference on test images
* Export saved YOLOX weights for future inference

### **About**

[Roboflow](https://roboflow.com) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

**Looking for a vision model available via API without hassle? Try Roboflow Train.**

![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)

# Install YOLOX Dependencies

In [ ]:
# Change the current working directory
!pwd
!nvidia-smi
%cd SwitchFrequencyAnalysis

In [ ]:
!git clone https://github.com/JiaPai12138/YOLOX.git
%cd YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  
!pip uninstall -y torch torchvision torchaudio
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

## Install Nvidia Apex

In [ ]:
%cd /content/
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

## Install PyCocoTools

In [ ]:
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# Export Trained Weights for Future Inference

Now that you have trained your custom detector, you can export the trained weights you have made here for inference on your device elsewhere

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list contents in yolo folder in your drive
!ls /mydrive/yolox

# Download or copy your Data

We'll download our dataset from Roboflow. Use the "**Pascal VOC**" export format.

To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).


In [ ]:
%cd SwitchFrequencyAnalysis
#%cd /content/
#!curl -L "[YOUR LINK HERE]" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

#%cd YOLOX/

#!ln -s /content/train/ ./datasets/VOCdevkit
#!ln -s /content/valid/ ./datasets/VOCdevkit
#!ln -s /content/test/ ./datasets/VOCdevkit

In [ ]:
%cd SwitchFrequencyAnalysis
%cd /content/

# copy the datasets zip file to the root darknet folder
!cp /mydrive/yolox/objx.zip ../

# unzip the datasets and their contents so that they are now in YOLOX/datasets/VOCdevkit/ folder
!unzip ../objx.zip -d YOLOX/datasets/VOCdevkit/

## Format Your Data Appropriately

In [ ]:
%cd SwitchFrequencyAnalysis
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "/content/YOLOX/datasets/VOCdevkit/"

## Change the Classes
Make sure you change the classes based on what your dataset. To ensure that the training process will function as intended, write the classes in lowercase with no whitespace.

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/YOLOX/yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  "human-head",
  "human-body"
)

In [ ]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
'''
%%writetemplate /content/YOLOX/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  "rbc",
  "wbc",
  "platelets"
)
'''

Set the number of classes you have in your dataset in te `NUM_CLASSES` variable

In [ ]:
NUM_CLASSES = 2
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

# Download Pretrained Weights

In [ ]:
%cd SwitchFrequencyAnalysis
%cd /content/
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.0/yolox_tiny.onnx
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.0/yolox_tiny.pth
%cd /content/YOLOX/

# Train the Model

In [ ]:
%cd SwitchFrequencyAnalysis
%cd /content/YOLOX/
!ls


In [ ]:
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_tiny.py -d 1 -b 64 --fp16 -o -c /content/yolox_tiny.pth

# Transfer the Model

In [ ]:
!python3 tools/export_onnx.py --output-name yolox_tiny.onnx -f exps/example/yolox_voc/yolox_voc_tiny.py --batch-size 1 -c best_ckpt.v2.pth